# 링크: http://bit.ly/35Tc1h3

# Python 버전 확인 및 OpenCV설치

In [0]:
!python --version

Python 3.6.8


In [0]:
!pip install -U -q PyDrive
# !pip install opencv-python==3.4.2.16 && pip install opencv-contrib-python==3.4.2.16

# OpenCV 버전 확인

In [0]:
import cv2
cv2.__version__

'3.4.3'

# 실습 이미지 준비

##구글 드라이브 연결
Colab의 현재 세션과 구글 드라이브를 연결하기 위해서 사용한다.

사용방법: 코드 실행시 오른쪽 GDrive_Auth 아래에 생성되는 링크를 타고 구글 로그인을 한뒤에 권한을 수락하고 보이는 코드를 복사하여 링크 아래의 사각형에 붙여넣은 후 엔터를 친다.

In [0]:
#@title GDrive_Auth { vertical-output: true, display-mode: "both" }

# Colab의 현재 세션과 구글 드라이브를 연결하기 위해서 사용한다.
# 파일를 저장하거나 불러오는데 필요하다.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## 비디오 다운로드

In [0]:
# 구글드라이브 공유 링크
link = 'https://drive.google.com/open?id=1yNBPIw_UQTly-pnCsikFs4McQjz_ywJ7'

# Download image zip file
file3 = drive.CreateFile({'id': link.split('=')[-1]})
print('Downloading file %s from Google Drive' % file3['title'])
file3.GetContentFile(file3['title'])  # Save Drive file as a local file


In [0]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('slow.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Optical Flow 연산

1. 이전 영상에서 keypoint 찾기
2. 현재 영상에서 keypoint 찾기
1. 서로 상응하는 keypoint의 상태값 1로 설정
3. 상태값이 1인 keypoint 정리
4. 이전 keypoint 와 현재 keypoint 연결로 가시화

In [0]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture('slow.mp4')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3, #threshold for eigenvalue
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2, # Pyramid image level
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

img_array = []
ret = True

while(True):
    ret,frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)
    img_array.append(img)
    
    # cv2_imshow(img)
    # k = cv2.waitKey(30) & 0xff
    # if k == 27:
    #     break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

# cv2.destroyAllWindows()
cap.release()


In [0]:
print(len(img_array))

height, width, layers = img_array[0].shape
size = (width,height)

out = cv2.VideoWriter('project.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 30, size)
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

915
